In [1]:
import pandas as pd 
import mlflow
import mlflow.sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [3]:
housing['target_names'], housing['feature_names']

(['MedHouseVal'],
 ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'])

In [4]:
## Preparing the dataset
data = pd.DataFrame(housing['data'], columns=housing['feature_names'])
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [5]:
data['target (price)'] = housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target (price)
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


### Train Test Split, Model Hyperparameter Tuning, MLFlow Experiments

In [6]:
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [7]:
# Hyperparameter tuning using Grid SearchCV

def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    return grid_search

In [8]:
X = data[housing['feature_names']]
y = data['target (price)']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
signature = infer_signature(X_train, y_train)

# Define the hyperparamtere grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split' : [2, 5],
    'min_samples_leaf': [1, 2]
}

In [9]:
## start the mlflow experiments
from mlflow import get_tracking_uri


with mlflow.start_run():
    # perform hyperparameter tuning
    grid_search=hyperparameter_tuning(X_train, y_train, param_grid)
    # get the best model
    best_model = grid_search.best_estimator_
    # Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # log best parameters
    mlflow.log_param('best_n_estimators', grid_search.best_params_['n_estimators'])
    mlflow.log_param('best_max_depth', grid_search.best_params_['max_depth'])
    mlflow.log_param('best_min_samples_split', grid_search.best_params_['min_samples_split'])
    mlflow.log_param('best_min_samples_leaf', grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric('mse', mse)

    ## Tracking url 
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000/')
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, 'model', registered_model_name='Best Random Forest Model')
    else:
        mlflow.sklearn.log_model(best_model, 'model', signature=signature)
    

    print('Best Hyperparameters : ', grid_search.best_params_)
    print('Mean Squared Error : ', mse)

2025/12/11 12:39:41 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/11 12:39:41 INFO mlflow.store.db.utils: Updating database tables
2025/12/11 12:39:41 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/11 12:39:41 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/11 12:39:42 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/11 12:39:42 INFO alembic.runtime.migration: Will assume non-transactional DDL.


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.6s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.8s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   6.9s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   3.4s
[CV] END max_depth=5, min_samples_leaf=

2025/12/11 12:41:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'Best Random Forest Model'.
2025/12/11 12:42:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Random Forest Model, version 1
Created version '1' of model 'Best Random Forest Model'.


Best Hyperparameters :  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error :  0.256358999283527
🏃 View run placid-deer-767 at: http://127.0.0.1:5000/#/experiments/0/runs/390e62d42a494064a443e50b016403db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
